# LT-Gate Evaluation Summary

This notebook provides a comprehensive analysis of the LT-Gate experiments, including:
1. Final metrics for all algorithms
2. Statistical significance tests
3. Training curves and visualizations
4. Ablation study results

In [ ]:
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Add project root to path
project_root = Path.cwd().parent
sys.path.append(str(project_root))

from analysis.eval_metrics import load_energy, load_training_metrics
from analysis.stats_tests import compare_algorithms
from analysis.plotting import barplot_metrics, plot_training_curves, plot_ablation_curves

## 1. Load Master Metrics

First, let's load and display the aggregated metrics from all algorithms.

In [ ]:
# Load master metrics CSV
df = pd.read_csv(project_root / 'logs/master_metrics.csv')

# Display metrics table
print("Final Metrics Summary:")
print("-" * 80)
display(df.style.format({
    'acc_task1': '{:.3f}',
    'acc_task2': '{:.3f}',
    'avg_accuracy': '{:.3f}',
    'forgetting': '{:.3f}',
    'energy_J': '{:.4f}',
    'latency_ms': '{:.2f}',
    'spikes_per_sample': '{:.1f}'
}))

## 2. Generate Metric Bar Plots

Create Figure 2 from the paper showing key metrics across algorithms.

In [ ]:
# Generate bar plots
metrics = ['acc_task2', 'forgetting', 'energy_J']
barplot_metrics(df, metrics=metrics)
plt.show()

## 3. Statistical Tests

Run paired t-tests between algorithms to assess significance of differences.

In [ ]:
# Compare LT-Gate vs baselines
metrics = ['energy_J', 'forgetting', 'acc_task2']
comparisons = [('ltgate', 'hlop'), ('ltgate', 'dsd')]

for metric in metrics:
    print(f"\nMetric: {metric}")
    print("-" * 40)
    
    for algo1, algo2 in comparisons:
        results = compare_algorithms(algo1, algo2, metric)
        print(f"\n{algo1.upper()} vs {algo2.upper()}:")
        print(f"Mean difference: {results['mean_difference']:.4f}")
        print(f"p-value: {results['p_value']:.2e}")
        print(f"Effect size (Cohen's d): {results['effect_size']:.3f}")
        print(f"Significant: {results['significant']}")

## 4. Training Curves

Plot accuracy and spike count evolution during training.

In [ ]:
# Get training CSV paths
csv_paths = {
    algo: project_root / f'logs/{algo}_train.csv'
    for algo in df['algorithm'].unique()
}

# Plot training curves
plot_training_curves(csv_paths)
plt.show()

## 5. Ablation Study

Analyze the effect of τ-ratio on forgetting and accuracy.

In [ ]:
# Load ablation results
ablation_df = pd.read_csv(project_root / 'logs/ltgate_ablate_tau.csv')

# Plot ablation curves
plot_ablation_curves(
    ablation_df,
    x='tau_ratio',
    metrics=['forgetting', 'acc_task2']
)
plt.show()